In [3]:
# Step 1: Install and import necessary libraries
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np





[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [14]:
# Step 2: Load text data
raw_text = gutenberg.raw('carroll-alice.txt')
raw_text = raw_text.lower().replace('\n', ' ')

print(raw_text[0:500])

[alice's adventures in wonderland by lewis carroll 1865]  chapter i. down the rabbit-hole  alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought alice 'without pictures or conversation?'  so she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy an


In [5]:
# Step 3: Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([raw_text])
total_words = len(tokenizer.word_index) + 1



In [6]:
# Step 4: Create input sequences
token_list = tokenizer.texts_to_sequences([raw_text])[0]
input_sequences = []
sequence_length = 5

for i in range(sequence_length, len(token_list)):
    input_sequences.append(token_list[i-sequence_length:i+1])

input_sequences = np.array(input_sequences)
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)




In [12]:
# Step 5: Build LSTM model
model = Sequential([
    Embedding(total_words, 64, input_length=sequence_length),
    LSTM(128),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=20, batch_size=256)

Epoch 1/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0503 - loss: 7.2933
Epoch 2/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0589 - loss: 6.1029
Epoch 3/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0586 - loss: 5.9789
Epoch 4/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0720 - loss: 5.8659
Epoch 5/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0834 - loss: 5.7734
Epoch 6/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0924 - loss: 5.6308
Epoch 7/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0967 - loss: 5.5186
Epoch 8/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1067 - loss: 5.3941
Epoch 9/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.1122 - loss: 5.3159
Epoch 10/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1194 - loss: 5.2077
Epoch 11/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1250 - loss: 5.1342
Epoch 12/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

In [16]:
# Step 6: Prediction function
def predict_next_word(seed_text, next_words=1):
    for _ in range(next_words):
        token_seq = tokenizer.texts_to_sequences([seed_text])[0]
        token_seq = pad_sequences([token_seq], maxlen=sequence_length, padding='pre')
        predicted = np.argmax(model.predict(token_seq, verbose=0), axis=-1)[0]
        seed_text += " " + tokenizer.index_word[predicted]
    return seed_text

# Example use
print(predict_next_word("alice was beginning", next_words=5))

alice was beginning to be a little voice
